Código sobre el Análisis de Fugas de Agua en la Ciudad de México 

In [1]:
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
# IMPORTAR PAQUETES NECESARIOS
import pandas as pd  # Para manipulación y análisis de datos tabulares
import requests  # Para realizar solicitudes HTTP a API o sitios web
import geopandas as gpd  # Para trabajar con datos geoespaciales (puntos, líneas, polígonos)
from shapely.geometry import Point, MultiPoint, Polygon
import json  # Para trabajar con datos JSON (formato de intercambio de datos

In [20]:
# Consumo de agua 
agua = 'https://datos.cdmx.gob.mx/dataset/57ffc13a-7207-4b69-a9a9-2fef3fce6331/resource/65a6b1a6-5d6e-49b9-aeed-ca7b22e8de03/download/reportes_agua_hist.csv'
agua = pd.read_csv(agua)
agua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254730 entries, 0 to 254729
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   folio                    254730 non-null  object 
 1   tipo_de_falla            254725 non-null  object 
 2   quien_atiende            254584 non-null  object 
 3   latitud                  254730 non-null  float64
 4   longitud                 254730 non-null  float64
 5   codigo_postal            238976 non-null  float64
 6   fecha                    254730 non-null  object 
 7   colonia_registro_sacmex  253535 non-null  object 
 8   colonia_datos_abiertos   250610 non-null  object 
 9   alcaldia                 250610 non-null  object 
dtypes: float64(3), object(7)
memory usage: 19.4+ MB


In [21]:
#Diccionario con los nombres de acaldías y su clave de municipio
alcaldia_d = {
    "AZCAPOTZALCO": "002",
    "COYOACAN": "003",
    "CUAJIMALPA DE MORELOS": "004",
    "GUSTAVO A. MADERO": "005",
    "IZTACALCO": "006",
    "IZTAPALAPA": "007",
    "LA MAGDALENA CONTRERAS": "008",
    "MILPA ALTA": "009",
    "ALVARO OBREGON": "010",
    "TLALPAN": "012",
    "XOCHIMILCO": "013",
    "BENITO JUAREZ": "014",
    "CUAUHTEMOC": "015",
    "MIGUEL HIDALGO": "016",
    "VENUSTIANO CARRANZA": "017",
    "TLAHUAC": "011",
}
agua["cve_mun"] = agua["alcaldia"].map(alcaldia_d)

In [22]:
##Crear cartografía con la longitud y latitud

# Crear la columna 'geometry' en el DataFrame original 'agua' con los puntos
agua['geometry'] = gpd.points_from_xy(agua['longitud'], agua['latitud'])

# Convertir 'agua' en un GeoDataFrame
agua = gpd.GeoDataFrame(agua, crs="EPSG:4326")

# Crear una función para generar polígonos a partir de los puntos agrupados por alcaldía
def crear_poligono(grupo):
    puntos = MultiPoint(list(grupo))  # Crear un conjunto de puntos (MultiPoint)
    return puntos.convex_hull  # Generar el polígono más simple (Convex Hull)

# Crear los polígonos por alcaldía
poligonos_por_alcaldia = (
    agua.groupby('alcaldia')['geometry']
    .apply(lambda x: crear_poligono(x))
    .reset_index(name='geometry_poligono')
)

# Añadir los polígonos al DataFrame original
agua = agua.merge(poligonos_por_alcaldia, on='alcaldia', how='left')

In [23]:
agua.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 254730 entries, 0 to 254729
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   folio                    254730 non-null  object  
 1   tipo_de_falla            254725 non-null  object  
 2   quien_atiende            254584 non-null  object  
 3   latitud                  254730 non-null  float64 
 4   longitud                 254730 non-null  float64 
 5   codigo_postal            238976 non-null  float64 
 6   fecha                    254730 non-null  object  
 7   colonia_registro_sacmex  253535 non-null  object  
 8   colonia_datos_abiertos   250610 non-null  object  
 9   alcaldia                 250610 non-null  object  
 10  cve_mun                  250610 non-null  object  
 11  geometry                 254730 non-null  geometry
 12  geometry_poligono        250610 non-null  geometry
dtypes: float64(3), geometry(2), object(8

In [24]:
print("Ver las fallas",agua["tipo_de_falla"].unique()) 

Ver las fallas ['Falta de agua' 'Mala calidad' 'Fuga' nan]


In [25]:
print("Ver las fechas",agua["fecha"].unique()) 

Ver las fechas ['2021-04-05' '2021-04-28' '2021-04-29' ... '2021-06-27' '2021-06-29'
 '2021-05-08']


In [30]:
# Columna 'fecha' sea de tipo datetime
agua['fecha'] = pd.to_datetime(agua['fecha'], errors='coerce')

# Filtrar los datos para el año 2021 y tipo de falla "Fuga"
agua_2021_fuga = agua[(agua['fecha'].dt.year == 2021) & (agua['tipo_de_falla'] == 'Fuga')]

# Crear una nueva columna 'mes' para extraer el mes de la fecha
agua_2021_fuga['mes'] = agua_2021_fuga['fecha'].dt.month

# Agrupar por mes y alcaldía y contar el número de folios
folio_por_mes = agua_2021_fuga.groupby(['mes', 'alcaldia']).size().reset_index(name='numero_folios')

# Crear la tabla pivote
pivot_folios = folio_por_mes.pivot_table(index='mes', columns='alcaldia', values='numero_folios', aggfunc='sum', fill_value=0)
pivot_folios


c:\Users\quetz\miniconda3\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


alcaldia,ALVARO OBREGON,AZCAPOTZALCO,BENITO JUAREZ,COYOACAN,CUAJIMALPA DE MORELOS,CUAUHTEMOC,GUSTAVO A. MADERO,IZTACALCO,IZTAPALAPA,LA MAGDALENA CONTRERAS,MIGUEL HIDALGO,MILPA ALTA,TLAHUAC,TLALPAN,VENUSTIANO CARRANZA,XOCHIMILCO
mes,,,,,,,,,,,,,,,,
1,425,135,129,608,44,118,545,72,290,112,104,32,91,277,41,149
2,313,94,116,445,44,57,425,70,243,78,88,11,73,234,57,118
3,238,83,74,292,26,39,367,40,147,53,107,8,29,217,43,109
4,306,77,72,299,36,47,427,28,211,61,103,20,95,216,66,144
5,229,77,71,269,44,39,394,32,203,62,82,25,60,199,67,116
6,127,37,31,176,22,12,219,17,131,26,27,30,71,123,24,86
7,109,38,23,145,12,26,228,29,89,32,44,22,25,101,30,77
8,819,144,121,397,82,83,472,76,536,291,144,21,124,361,70,215
9,579,213,195,546,54,116,557,114,389,194,192,12,145,368,89,215


In [31]:
# Los datos de los reportes de fuga de agua por alcaldía y mes
data = {
    'ALVARO OBREGON': [425, 313, 238, 306, 229, 127, 109, 819, 579, 566, 603, 404],
    'AZCAPOTZALCO': [135, 94, 83, 77, 77, 37, 38, 144, 213, 151, 133, 99],
    'BENITO JUAREZ': [129, 116, 74, 72, 71, 31, 23, 121, 195, 128, 140, 93],
    'COYOACAN': [608, 445, 292, 299, 269, 176, 145, 397, 546, 644, 558, 416],
    'CUAJIMALPA DE MORELOS': [44, 44, 26, 36, 44, 22, 12, 82, 54, 77, 44, 46],
    'CUAUHTEMOC': [118, 57, 39, 47, 39, 12, 26, 83, 116, 79, 80, 118],
    'GUSTAVO A. MADERO': [545, 425, 367, 427, 394, 219, 228, 472, 557, 463, 519, 537],
    'IZTACALCO': [72, 70, 40, 28, 32, 17, 29, 76, 114, 50, 65, 62],
    'IZTAPALAPA': [290, 243, 147, 211, 203, 131, 89, 536, 389, 385, 363, 383],
    'LA MAGDALENA CONTRERAS': [112, 78, 53, 61, 62, 26, 32, 291, 194, 250, 157, 145],
    'MIGUEL HIDALGO': [104, 88, 107, 103, 82, 27, 44, 144, 192, 191, 253, 167],
    'MILPA ALTA': [32, 11, 8, 20, 25, 30, 22, 21, 12, 8, 11, 10],
    'TLAHUAC': [91, 73, 29, 95, 60, 71, 25, 124, 145, 110, 78, 73],
    'TLALPAN': [277, 234, 217, 216, 199, 123, 101, 361, 368, 534, 460, 412],
    'VENUSTIANO CARRANZA': [41, 57, 43, 66, 67, 24, 30, 70, 89, 64, 64, 49],
    'XOCHIMILCO': [149, 118, 109, 144, 116, 86, 77, 215, 215, 196, 190, 224],
}

# Convertir los datos a un DataFrame
df = pd.DataFrame(data)

# Calcular la matriz de transición
transition_matrix = pd.DataFrame(index=df.columns, columns=df.columns, dtype=float)

# Calcular las probabilidades de transición
for column in df.columns:
    for row in df.columns:
        transition_matrix.at[row, column] = df[column].shift(-1).dropna().div(df[column].sum(), axis=0).mean()

# Mostrar la matriz de transición
print("Matriz de transición:")
print(transition_matrix)

# Predicciones para los próximos años (se calculan con la matriz de transición)
predicciones = pd.DataFrame(index=[f'Año {i+1}' for i in range(5)], columns=df.columns)

# Empezar con el primer año como el valor actual (o último año de datos)
predicciones.iloc[0] = df.iloc[-1]

# Hacer predicciones para los próximos 5 años
for i in range(1, 5):
    predicciones.iloc[i] = predicciones.iloc[i-1].dot(transition_matrix)

print("\nPredicciones para los próximos 5 años:")
predicciones

Matriz de transición:
                        ALVARO OBREGON  AZCAPOTZALCO  BENITO JUAREZ  COYOACAN  \
ALVARO OBREGON                 0.08272      0.081329       0.081079  0.079382   
AZCAPOTZALCO                   0.08272      0.081329       0.081079  0.079382   
BENITO JUAREZ                  0.08272      0.081329       0.081079  0.079382   
COYOACAN                       0.08272      0.081329       0.081079  0.079382   
CUAJIMALPA DE MORELOS          0.08272      0.081329       0.081079  0.079382   
CUAUHTEMOC                     0.08272      0.081329       0.081079  0.079382   
GUSTAVO A. MADERO              0.08272      0.081329       0.081079  0.079382   
IZTACALCO                      0.08272      0.081329       0.081079  0.079382   
IZTAPALAPA                     0.08272      0.081329       0.081079  0.079382   
LA MAGDALENA CONTRERAS         0.08272      0.081329       0.081079  0.079382   
MIGUEL HIDALGO                 0.08272      0.081329       0.081079  0.079382   
MILPA 

,ALVARO OBREGON,AZCAPOTZALCO,BENITO JUAREZ,COYOACAN,CUAJIMALPA DE MORELOS,CUAUHTEMOC,GUSTAVO A. MADERO,IZTACALCO,IZTAPALAPA,LA MAGDALENA CONTRERAS,MIGUEL HIDALGO,MILPA ALTA,TLAHUAC,TLALPAN,VENUSTIANO CARRANZA,XOCHIMILCO
Año 1,404,99,93,416,46,118,537,62,383,145,167,10,73,412,49,224
Año 2,267.8472,263.341707,262.533872,257.038696,269.971923,251.691758,263.230669,262.006107,269.032641,271.797772,273.9816,249.508225,266.86149,271.080162,276.187568,270.513619
Año 3,351.280611,345.371674,344.312201,337.105297,354.067177,330.092808,345.226047,343.620039,352.835312,356.461772,359.325854,327.229113,349.987855,355.520629,362.218973,354.77761
Año 4,460.703221,452.953672,451.564176,442.112349,464.357791,432.915496,452.762683,450.656409,462.742206,467.498294,471.254527,429.15977,459.007776,466.263988,475.048842,465.289521
Año 5,604.210568,594.047063,592.224745,579.828709,609.003524,567.76707,593.796582,591.034211,606.884689,613.122281,618.048567,562.841449,601.986999,611.503493,623.024797,610.225484
